In [ ]:
from helper import *

path='../reports/corpus.xlsx'

description = get_column_values(path, 'descriptions_measures', 'A')


targets = get_column_values(path, 'descriptions_measures', 'B')

similarities_df = similarities_df(docs=description,group=targets,encoding_method='tfidf')

similarities_df.head()

In [ ]:
from helper import *

Q1 = 'Create a report that shows the number of settlement applications approved during the month, their approved amount,\
      the written off balance, the average days to approval, their average and median duration and their entry principal and balance, \
        for September 2020 per DCA and application type. The report should be produced on Earth portfolio. '

path = '../reports/corpus.xlsx'
targets = get_column_values(path, 'descriptions_measures', 'B')
encoding_method = 'word2vec'
version = 'word2vec-google-news-300'




similarities_dict = get_similarity_scores(text=Q1,documents=targets,encoding_method=encoding_method,version=version)

top_10 = get_first_n_keys(similarities_dict, 10)


top_10

In [ ]:
from helper import *

Q1 = 'Create a report that shows the number of settlement applications approved during the month, their approved amount,\
      the written off balance, the average days to approval, their average and median duration and their entry principal and balance, \
        for September 2020 per DCA and application type. The report should be produced on Earth portfolio. '

processed_text = preprocess_text(Q1)



w2v = api.load("word2vec-google-news-300")
vocab = w2v.index_to_key
words = Q1.split()


count = sum(1 for word in processed_text if word in w2v)
existing_words = [word for word in processed_text if word in w2v]
difference = len(words) - count

pr, orig = get_unique_items(words, processed_text)

pr

In [5]:
from helper import *

path='../reports/corpus.xlsx'

Q1 = 'Create a report that shows the number of settlement applications approved during the month, their approved amount,\
      the written off balance, the average days to approval, their average and median duration and their entry principal and balance, \
        for September 2020 per DCA and application type. The report should be produced on Earth portfolio. '

Q2 = 'Provide a report that displays the number of accounts not in running settlements for the Earth portfolio, \
  their expected monthly payments at the end of the examined month and their actual monthly payments per DCA, for September 2020.'

ground_truth = {'R1':[preprocess_text(element) for element in get_column_values(path, 'R1', 'C')],
                        'R2':[preprocess_text(element) for element in get_column_values(path, 'R2', 'C')],
                        'R3':[preprocess_text(element) for element in get_column_values(path, 'R3', 'C')],
                        'R4':[preprocess_text(element) for element in get_column_values(path, 'R4', 'C')],
                        'R5':[preprocess_text(element) for element in get_column_values(path, 'R5', 'C')],
                        'R6':[preprocess_text(element) for element in get_column_values(path, 'R6', 'C')],
                        'R7':[preprocess_text(element) for element in get_column_values(path, 'R7', 'C')],
                        'R8':[preprocess_text(element) for element in get_column_values(path, 'R8', 'C')],
                        'R9':[preprocess_text(element) for element in get_column_values(path, 'R9', 'C')]}   

processed_text = preprocess_text(Q2)

ground_truth['R4']


['settlement duration 6 month included',
 'settlement duration 7 12 month included',
 'settlement duration 13 36 month included',
 'settlement duration 37 72 month included',
 'settlement duration 73 108 month included',
 'settlement duration 109 month']

In [1]:
from helper import *

path='../reports/corpus.xlsx'
descriptions = get_column_values(path, 'descriptions_measures', 'A')
targets = get_column_values(path, 'descriptions_measures', 'B')
models = ['bow','tfidf','word2vec','sbert']
version_w2v = 'word2vec-google-news-300'
version_sbert = 'bert-base-nli-mean-tokens'
version_bow = '/models/bow_model.joblib'
version_tfidf = '/models/tfidf_model.joblib'
corpus = ['Descriptions']
tm = TextModels(excel_path=path, columns=corpus,
                 word2vec_version=version_w2v, sbert_model=version_sbert)

tm.build_bow_model(preprocessing=True)
tm.build_tfidf_model(preprocessing=True)

tm.save_models('preprocessed_nums_')

c:\Users\avitsas\Anaconda3\envs\project\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Bag-of-words model saved to preprocessed_nums_bow_model.joblib
TF-IDF model saved to preprocessed_nums_tfidf_model.joblib


In [6]:
from helper import *

path='../reports/corpus.xlsx'
descriptions = [preprocess_text(element) for element in get_column_values(path, 'descriptions_measures', 'A')]
targets = [preprocess_text(element) for element in get_column_values(path, 'descriptions_measures', 'B')]
models = ['bow','tfidf','word2vec','sbert']
preprocessing = True
version_w2v = 'word2vec-google-news-300'
version_sbert = 'bert-base-nli-mean-tokens'
version_bow = '/models/preprocessedbow_model.joblib'
version_tfidf = '/models/preprocessedtfidf_model.joblib'
corpus = ['Descriptions']
tm = TextModels(excel_path=path, columns=corpus,
                 word2vec_version=version_w2v, sbert_model=version_sbert)
n = 10

df = results_to_targets(descriptions=descriptions,targets=targets,model='bow',version=version_bow,
                        tm=tm, preprocessing=preprocessing,n=n,path=path, i=5)


dictsim = get_similarities_for_values(values=descriptions, docs=targets,encoding_method='bow',version=version_bow,
                                      tm=tm, n=n)

dictsim['R4']

['settlement duration 109 month',
 'settlement duration 6 month included',
 'settlement duration 7 12 month included',
 'settlement duration 13 36 month included',
 'settlement duration 37 72 month included',
 'settlement duration 73 108 month included',
 'month arrears 1 month ago',
 'application status previous month',
 'settlement duration',
 'payment prediction stray payer active settlement']

In [1]:
from helper import *

def w2v_vec(text):
    text = text.split()
    embeddings = [w2v[word] for word in words if word in w2v]
    sentence_emb = sum(embeddings)/w2v.vector_size
    return sentence_emb

def cos_sim(v1,v2):
  score = cosine_similarity([v1], [v2])[0][0]
  return score
    
path='../reports/corpus.xlsx'
descriptions = get_column_values(path, 'descriptions_measures', 'A')
targets = get_column_values(path, 'descriptions_measures', 'B')
models = ['bow','tfidf','word2vec','sbert']
preprocessing = False
version_w2v = 'glove-twitter-200'
version_sbert = 'bert-base-nli-mean-tokens'
version_bow = 'preprocessedbow_model.joblib'
version_tfidf = 'preprocessedtfidf_model.joblib'
corpus = ['Descriptions']
tm = TextModels(excel_path=path, columns=corpus,
                 word2vec_version=version_w2v, sbert_model=version_sbert)
n = 10
Q8 = 'Create a report that shows the number of settlement applications approved during the month, their approved amount,\
      the written off balance, the average days to approval, their average and median duration and their entry principal and balance, \
        for September 2020 per DCA and application type. The report should be produced on Earth portfolio. '
target_sample = 'Balance on arrangement creation'


scores = get_similarity_scores(text=Q8, documents=targets, encoding_method='word2vec',
                               version=version_w2v, tm=tm, preprocessing=preprocessing )


scores

c:\Users\avitsas\Anaconda3\envs\project\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'Status of the last submitted application within the observation period.': 0.9315781,
 'Last rejection date of all applications within the observation perion': 0.9308385,
 'Application status at the end of the preceding observation period': 0.92961,
 'Payment prediction for stray payers, not in active settlement': 0.9096353,
 'Application type in the observation period': 0.9019604,
 'Application bucket in the previous observation period': 0.9003092,
 'External agency (DCA or legal office) assigned on previous observation point': 0.89811665,
 'Amount paid within the observation period': 0.8942149,
 'Bucket class for active arrangements or cancelled status': 0.8844932,
 'Application type in previous observation period': 0.85671395,
 'Application status in previous month': 0.85315996,
 'Months in arrears, 1 month ago': 0.8467916,
 'Total payments amount so far': 0.83751345,
 'Settlement duration more than 109 months': 0.8333211,
 'External agency (DCA or legal office) currently assigned'

In [ ]:
glove_Q8 = tm.encode_word2vec(sentence=Q8, preprocessing=preprocessing)
glove_target_sample = tm.encode_word2vec(sentence=target_sample, preprocessing=preprocessing)



glove_Q8

In [3]:
def w2v_vec(text,w2v):
    text = text.split()
    embeddings = [w2v[word] for word in text if word in w2v]
    sentence_emb = sum(embeddings)/w2v.vector_size
    return sentence_emb

def cos_sim(t1,t2,w2v):
  v1 = w2v_vec(t1,w2v)
  v2 = w2v_vec(t2,w2v)
  score = cosine_similarity([v1], [v2])[0][0]
  return score

w2v = api.load("glove-twitter-200")
Q8 = 'Create a report that shows the number of settlement applications approved during the month, their approved amount,\
      the written off balance, the average days to approval, their average and median duration and their entry principal and balance, \
        for September 2020 per DCA and application type. The report should be produced on Earth portfolio. '
target_sample = 'Balance on arrangement creation'


sim = cos_sim(Q8,target_sample,w2v)

query_encoding = w2v_vec(target_sample, w2v) 
doc_encoding = w2v_vec(Q8, w2v)
similarity_score = cosine_similarity([query_encoding], [doc_encoding])[0][0]

print('doc_encoding shape:', doc_encoding.shape)





[-3.7266901e-03 -5.2159498e-03  8.9269993e-04  4.1858002e-04
  6.2018000e-03  4.0180152e-03  6.3200500e-03 -5.1428499e-03
 -2.4992495e-04 -4.5147650e-03 -4.9845502e-03  4.4995494e-04
 -9.8515498e-03  9.8384998e-04 -4.2974604e-03  3.5630499e-03
  1.4208248e-03  3.6028496e-04 -4.0710745e-03  2.6349584e-03
  1.6883650e-03  1.6630000e-03 -4.5094001e-03 -2.0826503e-03
 -1.2412150e-03  1.4283050e-02  1.8301500e-03  6.9146003e-03
  2.1180550e-03  2.3239499e-03 -1.1422550e-03  3.0653001e-04
 -2.5102999e-03 -2.4493001e-03  5.0821993e-03 -3.5678900e-03
  3.3366247e-03  5.1048503e-04  5.8827000e-03 -5.4690003e-04
  7.5462502e-03 -1.5127101e-03  9.9425006e-04  4.9095000e-03
  5.3505506e-03 -1.9670010e-04 -7.2230948e-03 -2.4558499e-03
  1.5668500e-03 -5.2830498e-03  2.2397500e-03 -3.0819996e-04
 -6.2564999e-04  2.2696999e-03 -7.0864498e-04 -1.3213800e-03
  4.8332498e-03 -2.8948497e-03  1.1388500e-03  8.9293451e-04
 -8.9122995e-04 -3.9040498e-04  8.9254498e-04 -4.0206448e-03
 -2.9283601e-03  3.08945

In [9]:
from helper import *

path='../reports/corpus.xlsx'
descriptions = get_column_values(path, 'descriptions_measures', 'A')
targets = get_column_values(path, 'descriptions_measures', 'B')
w2v = 'word2vec-google-news-300'

dict_desc = percentage_of_words_in_word2vec_vocabulary(w2v,descriptions)
dict_tar = percentage_of_words_in_word2vec_vocabulary(w2v,targets)


In [22]:
#dict_desc
dict_tar
w2v = api.load('word2vec-google-news-300')
l = 'QC customer code'.split()

words_found = 0
for word in l:
    if word in w2v:
        words_found += 1

words_found

'QC' in w2v

True

In [1]:
from helper import *

path='../reports/corpus.xlsx'
descriptions = get_column_values(path, 'descriptions_measures', 'A')
targets = get_column_values(path, 'descriptions_measures', 'B')
model = 'bow'
preprocessing = False
version_w2v = 'word2vec-google-news-300'
version_sbert = 'bert-base-nli-mean-tokens'
version_bow = './models/bow_model.joblib'
version_tfidf = './models/tfidf_model.joblib'
corpus = ['Descriptions']
tm = TextModels(excel_path=path, columns=corpus,
                 word2vec_version=version_w2v)
n = 10
metric = 'precision'
results = get_similarities_for_values(values=descriptions, docs=targets, encoding_method=model,
                                      version=version_bow, tm=tm, preprocessing=preprocessing, n=n)



results

c:\Users\avitsas\Anaconda3\envs\project\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'Q1': ['Application status at the end of the preceding observation period',
  'Status of the last submitted application within the observation period.',
  'Last rejection date of all applications within the observation perion',
  'Application type in the observation period',
  'Amount paid within the observation period',
  'Application bucket in the previous observation period',
  'Portfolio',
  'Payment prediction for settlement payers',
  'Total payments amount so far',
  'Portfolio grouping'],
 'Q2': ['Application status at the end of the preceding observation period',
  'Status of the last submitted application within the observation period.',
  'Application type in the observation period',
  'Last rejection date of all applications within the observation perion',
  'Application bucket in the previous observation period',
  'Amount paid within the observation period',
  'Payment prediction for stray payers, not in active settlement',
  'Portfolio',
  'Application status in previou

In [1]:
from helper import *

path='../reports/corpus.xlsx'
descriptions = get_column_values(path, 'descriptions_measures', 'A')
targets = get_column_values(path, 'descriptions_measures', 'B')
model = 'bow'
preprocessing = False
version_w2v = 'word2vec-google-news-300'
version_sbert = 'bert-base-nli-mean-tokens'
version_bow = './models/bow_model.joblib'
version_tfidf = './models/tfidf_model.joblib'
corpus = ['Descriptions']
tm = TextModels(excel_path=path, columns=corpus,
                 word2vec_version=version_w2v)
n = 10

pr = precision(values=descriptions, docs=targets, encoding_method=model,
               version=version_bow, tm=tm, preprocessing=preprocessing, path=path, n=n)

pr

c:\Users\avitsas\Anaconda3\envs\project\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'Q1': 20.0,
 'Q2': 20.0,
 'Q3': 20.0,
 'Q4': 60.0,
 'Q5': 'One or both of the lists is empty',
 'Q6': 10.0,
 'Q7': 10.0,
 'Q8': 30.0,
 'Q9': 20.0}